In [1]:
%run functions.ipynb

Populating the interactive namespace from numpy and matplotlib


In [2]:
labels = load_file('data/labels_for_microarray_data.csv')
dataset = load_file('data/microarray_data.csv')[:, 1:].T
print(labels.shape, dataset.shape)

(969,) (969, 12179)


In [3]:
X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.25, random_state = random_state)
print("Train and test sizes: {} {}".format(X_train.shape, X_test.shape))
print("(1, 0) labels count in train test: {} {}".format((np.count_nonzero(y_train==1), np.count_nonzero(y_train==0)), 
                                                        (np.count_nonzero(y_test==1), np.count_nonzero(y_test==0))))

Train and test sizes: (726, 12179) (243, 12179)
(1, 0) labels count in train test: (289, 437) (104, 139)


In [4]:
#Save splitted data for varclust
def save_data(data, name):
    np.savetxt(name, np.vstack((np.arange(data.shape[-1]), data)), delimiter=',', fmt='%.13f')

def save_labels(data, name):
    np.savetxt(name, np.vstack((np.arange(data.shape[-1]), data)), delimiter=',', fmt='%i')
    
save_data(X_train, "data/microarray_train.csv")
save_labels(y_train, "data/labels_train.csv")
save_data(X_test, "data/microarray_test.csv")
save_labels(y_test,"data/labels_test.csv")

## Data preparation for SAM

In [5]:
dataset = pd.read_csv('data/microarray_data.csv', delimiter=",", index_col=0)
labels = load_file('data/labels_for_microarray_data.csv') + 1

In [6]:
result = pd.DataFrame(dataset)
result.columns = np.array(labels, dtype=np.int32)

In [7]:
result.head()

,2,2,2,2,2,2,2,2,2,2,...,1,1,1,1,1,1,1,1,1,1
GeneSymbol,,,,,,,,,,,,,,,,,,,,,
RPL41,14.028,13.893,14.201,14.201,14.201,13.796,14.028,14.201,13.949,13.841,...,13.841,13.525,13.491,13.564,13.712,13.182,13.615,13.893,13.796,14.028
EEF1A1,13.321,14.028,13.893,13.615,13.341,13.712,13.459,13.712,13.893,13.224,...,14.028,13.615,13.615,13.615,14.028,14.028,13.752,14.028,13.949,14.201
RPL37A,13.287,13.491,13.564,13.287,13.432,13.406,13.615,13.383,13.491,13.182,...,13.673,13.155,13.084,13.304,13.383,13.752,13.182,13.564,13.712,13.459
TPT1,13.383,13.432,13.341,13.270,13.321,13.041,13.224,13.224,13.432,13.383,...,13.119,13.254,13.525,13.239,13.752,13.254,13.210,13.949,13.752,13.796
RPL23A,13.752,13.459,13.752,13.673,13.949,13.841,13.673,13.841,13.564,13.491,...,13.168,13.096,12.971,13.084,12.787,12.728,12.787,13.073,13.341,13.304


In [8]:
X_train, X_test = train_test_split(result.T, test_size=0.25, random_state = random_state)
X_train = X_train.T
X_train.head()

,1,1,2,1,1,2,2,2,1,1,...,2,1,2,1,2,2,2,1,1,2
GeneSymbol,,,,,,,,,,,,,,,,,,,,,
RPL41,13.432,14.028,13.949,13.752,13.949,13.564,13.893,13.406,13.949,14.028,...,14.028,13.893,13.893,13.893,13.615,13.406,13.615,13.362,13.615,13.712
EEF1A1,13.752,13.949,13.130,13.712,13.406,13.615,13.796,13.564,13.383,13.459,...,13.525,13.673,13.383,13.841,13.673,13.525,13.796,13.796,14.028,13.673
RPL37A,13.949,13.796,13.893,13.073,13.073,13.270,13.491,13.383,13.796,13.673,...,13.673,13.383,13.564,13.712,13.270,13.712,13.143,13.210,13.341,13.459
TPT1,13.841,13.525,13.459,14.028,13.796,13.224,13.287,13.459,13.459,13.796,...,13.491,13.712,13.406,13.459,13.341,13.673,13.406,13.195,12.898,13.564
RPL23A,13.084,13.712,13.383,13.254,13.210,13.168,13.525,13.491,13.432,13.841,...,13.615,13.224,13.525,13.406,12.952,12.916,13.893,12.453,13.168,13.051


In [9]:
X_train.insert(0, '', result.index.values)
X_train.insert(0, '', np.arange(result.shape[0]), allow_duplicates=True)

In [10]:
X_train.head()

,,,1,1,2,1,1,2,2,2,...,2,1,2,1,2,2,2,1,1,2
GeneSymbol,,,,,,,,,,,,,,,,,,,,,
RPL41,0,RPL41,13.432,14.028,13.949,13.752,13.949,13.564,13.893,13.406,...,14.028,13.893,13.893,13.893,13.615,13.406,13.615,13.362,13.615,13.712
EEF1A1,1,EEF1A1,13.752,13.949,13.130,13.712,13.406,13.615,13.796,13.564,...,13.525,13.673,13.383,13.841,13.673,13.525,13.796,13.796,14.028,13.673
RPL37A,2,RPL37A,13.949,13.796,13.893,13.073,13.073,13.270,13.491,13.383,...,13.673,13.383,13.564,13.712,13.270,13.712,13.143,13.210,13.341,13.459
TPT1,3,TPT1,13.841,13.525,13.459,14.028,13.796,13.224,13.287,13.459,...,13.491,13.712,13.406,13.459,13.341,13.673,13.406,13.195,12.898,13.564
RPL23A,4,RPL23A,13.084,13.712,13.383,13.254,13.210,13.168,13.525,13.491,...,13.615,13.224,13.525,13.406,12.952,12.916,13.893,12.453,13.168,13.051


In [11]:
X_train.to_excel('data/microarray_train_for_SAM.xlsx', index=False)